# Make Cube Network

In [ ]:
import os
import sys
import yaml
import pickle
from subprocess import Popen

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [ ]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network', 'network_standard', 'pickles')
output_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network', 'network_standard', 'cube')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [ ]:
variables_in_shape_file = [
                    "model_link_id",
                    "A",
                    "B",
                    "shstGeometryId",
                    "distance",
                    "roadway",
                    "name",
                    "roadway_class",
                    "bike_access",
                    "transit_access",
                    "walk_access",
                    "drive_access",
                    "truck_access",
                    "lanes_AM",
                    "lanes_MD",
                    "lanes_PM",
                    "lanes_NT",
                    "assign_group",
                    "centroidconnect",
                    "model_node_id",
                    "N",
                    "osm_node_id",
                    "bike_node",
                    "transit_node",
                    "walk_node",
                    "drive_node",
                    "geometry",
                    "X",
                    "Y",
                ]

parameters = Parameters(lasso_base_dir = lasso_dir,
                       output_variables = variables_in_shape_file)

## Read Roadway and Transit Networks

In [ ]:
roadway_net_filename = os.path.join(input_dir, 'roadway_network.pickle')
roadway_net = pickle.load(open(roadway_net_filename, 'rb'))

In [ ]:
# transit_net_filename = os.path.join(input_dir, 'transit_network.pickle')
# transit_net = pickle.load(open(transit_net_filename, 'rb'))

## Make Travel Model Networks

In [ ]:
model_roadway_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = roadway_net, 
    parameters = parameters,
)
model_roadway_net.roadway_standard_to_met_council_network()

## Write Networks to Disk as SHP and TXT

In [ ]:
# Approach A: only A and B node numbers written to the shape file 

# model_roadway_net.write_roadway_as_shp(
#     output_link_shp = os.path.join(output_dir, 'links.shp'),
#     output_node_shp = os.path.join(output_dir, 'nodes.shp'),
#     output_link_csv = os.path.join(output_dir, 'links.csv'),
#     output_node_csv = os.path.join(output_dir, 'nodes.csv'),
# )

In [ ]:
# Approach B: variables identified in the variables_in_shape_file list written to shape file
model_roadway_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    output_link_csv = os.path.join(output_dir, 'links.csv'),
    output_node_csv = os.path.join(output_dir, 'nodes.csv'),
    data_to_dbf = True,
)

In [ ]:
model_roadway_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links_do.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes_do.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    drive_only = True,
)

In [ ]:
# standard_transit_net = StandardTransit.fromTransitNetwork(transit_net)
# standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin"))

## Make Cube .net File

In [ ]:
os.chdir(output_dir)
p = Popen("make_cube.bat")
p.communicate()